## Population Lecture II



### Preface



Some packages we need to have installed (skip this if already installed):



In [1]:
!pip install plotly
!pip install wbdata
!pip install cufflinks

### Malthus



Among all the things ever written by social scientists (or
    &ldquo;political philosophers&rdquo;, to use a term he would have recognized),
    Malthus&rsquo; [&ldquo;Essay on Population&rdquo; (1798)](https://oll.libertyfund.org/titles/malthus-an-essay-on-the-principle-of-population-1798-1st-ed) must be among the most
    influential.  You can find the first edition at the link above.
    The main argument is spelled out in Chapter 1, which I encourage
    you to read.  A key point of the essay begins with [two postulata](https://oll.libertyfund.org/titles/311#Malthus_0195_20):

-   **First:** That food is necessary to the existence of man.
-   **Secondly:** That the passion between the sexes is necessary and
    will remain nearly in its present state.

These lead to a conclusion:

>     Assuming then, my postulata as granted, I say, that the power of
>     population is indefinitely greater than the power in the earth to
>     produce subsistence for man.  Population, when unchecked, increases
>     in a geometrical ratio. Subsistence increases only in an
>     arithmetical ratio. A slight acquaintance with numbers will shew
>     the immensity of the first power in comparison of the second.



#### Malthus meets math



Malthus&rsquo; argument boils down to a mathematical assertion: that
    population grows at an exponential rate, while food production
    increases only linearly.  We might write this as
    $$
       \mbox{Population}_t = \mbox{Population}_0e^{rt},
    $$
    where $r$ is the rate of population growth, and $t$ is the number
    of years that have passed; and

$$
       \mbox{Food}_t = \mbox{Food}_0 + mt.
    $$
 If we lack a &ldquo;slight acquaintance&rdquo; with numbers a graph makes the
 point:



In [1]:
from plotly.offline import init_notebook_mode, iplot
import numpy as np
from scipy.optimize import newton

init_notebook_mode(connected=True)

# Play with these constants
## for exponential
P0 = 1
r = 0.02

## for line
F0 = 10
m = 100

food_production = lambda t,m=m: F0+m*t

population = lambda t,r=r: P0*np.exp(r*t)

diff = lambda t,r=r,m=m: food_production(t,m) - population(t,r)

# Point where curves cross

def choose_xaxis(r,m,x0=10):
    """
    Find a range of x so that curves intersect
    """
    tstar = -1
    try:
        while tstar <= 0:
            tstar = newton(lambda t: diff(t,r,m),x0,maxiter=1000)
            x0 = x0*10
        return np.linspace(0,tstar*1.1,100)
    except RuntimeError: # Failed to find root
        return np.linspace(0,1000,100)

In [1]:
import plotly.graph_objs as go
from ipywidgets import interactive, HBox, VBox

T = choose_xaxis(r,m)

data = [dict(name='Food production',
             #visible = False,
             line=dict(color='blue', width=6),
             x = T,
             y = [food_production(t) for t in T]),
        dict(name='Population',
             #visible = False,
             line=dict(color='red', width=6),
             x = T,
             y = [population(t) for t in T])]

layout = dict()

#fig = dict(data=data, layout=layout)
#plot(fig)

f = go.FigureWidget(data=data,layout=layout)

def update_pop(r,m):
    T = choose_xaxis(r,m)
    f.data[1].y = [population(t,r) for t in T]
    f.data[0].x = T
    f.data[1].x = T

r_slider = interactive(update_pop,r=(0.0,0.08,.01),m=(0,500,20))
vb = VBox((f, r_slider))
vb.layout.align_items = 'center'
vb

### Food Production



Malthus thought that population would have to increase at an
   exponential rate.  We&rsquo;ve seen that in fact the rate of population
   increase has been roughly linear over the last six decades. (There
   are no guarantees about the future!)

Malthus also thought that food production (&ldquo;subsistence&rdquo;) could
only increase linearly (&ldquo;arithmetically&rdquo;).  We&rsquo;ll next take a look
to see what the data say about this.



#### Preface



In [1]:
import numpy as np
import wbdata
import cufflinks as cf
cf.go_offline()

def fix_date_index(df):
    idx_vars = df.index.names
    new = df.reset_index()
    new.date = new.date.astype(int)
    return new.set_index(idx_vars)

#### Finding data



Maybe there&rsquo;s something relevant to food production in the WDI
 indicators?



In [1]:
wbdata.get_topic()

&ldquo;Agriculture & Rural Development&rdquo; sounds relevant.  Let&rsquo;s see what
 variables are available:



In [1]:
wbdata.get_indicator(topic=1)

#### Cereal production & land



The great majority of food (as measured by calories) originates in the
 form of cereals; some of these are turned into products directly
 consumed by humans (bread, rice, hominy); others are used to feed
 livestock, which humans subsequently eat.  So what&rsquo;s happened to
 cereal production over the last few decades?



In [1]:
vars = {"AG.PRD.CREL.MT":"Cereal production"}


use=["CHN","USA","IND","FRA","RUS","WLD"]

cereals = fix_date_index(wbdata.get_dataframe(vars,country=use)).squeeze().unstack('country')

# Use this to find top producers:
#big_producers = cereals.query('date==2020').squeeze().sort_values(ascending=False)
#big_producers.head(20)

cereals.iplot(xTitle="Year",yTitle="Metric Tons",
              title="Cereal Production by Top 5 (in 2020)")

#### FAO Food Index



The FAO compiles a series of data meant to measure changes in the
    supply of food of all sorts, including both plant and animal
    sources.  

From this we don&rsquo;t get levels; instead the series is normalized to an
arbitrary year.  Let&rsquo;s let each series take the value of 100 in the
year 2000, and compare this with a similarly normalized description of
world population:



In [1]:
vars = {"AG.PRD.LVSK.XD":"Livestock index",
        "AG.PRD.FOOD.XD":"Food index",
        "AG.PRD.CROP.XD":"Crop index",
        "SP.POP.TOTL":"Population"}

food = fix_date_index(wbdata.get_dataframe(vars)).dropna()
# Add population back...
Population = food.Population

# Weight indices by population
food = food.filter(regex='index$').multiply(Population,axis=0)

# No "WLD" or other regions; add up all countries
food = food.groupby('date').sum().replace(0,np.nan).dropna()


food.sort_index(inplace=True)

# Normalize so 2000 = 1
food = food/food.loc[2000,:]

food.iplot(xTitle="Year",yTitle="Index")

And here in growth rates:



In [1]:
np.log(food).diff().iplot(xTitle="Year",yTitle="Growth rates")